In [13]:
from time import sleep
from tqdm import tqdm

import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

from keys import GOOG_API_KEY

In [14]:
## Load dataframes
# Polling averages
pres = pd.read_csv('averages/presidential_gen_approval.csv')
pres_issue = pd.read_csv('averages/presidential_issue_approvals.csv')
pres_rv = pd.read_csv('averages/presidential_RV_approval.csv')

# Polls display tables
pres_polls = pd.read_csv('transformed_tables/gen_approval_polls.csv')
issue_polls = pd.read_csv('transformed_tables/issue_approval_polls.csv')

In [15]:
# Define the scope
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Authenticate with the service account
credentials = Credentials.from_service_account_file(
    GOOG_API_KEY,
    scopes=scopes
)

# Authorize gspread
client = gspread.authorize(credentials)

## Polling Averages

In [16]:
spreadsheet = client.open("snoutcounter-data")
worksheet = spreadsheet.worksheet("approval-stats")
worksheet.clear()

set_with_dataframe(worksheet, pres.reset_index())

In [17]:
worksheet = spreadsheet.worksheet("approval_stats_RV")
worksheet.clear()

set_with_dataframe(worksheet, pres_rv.reset_index())

In [18]:
issues = ['economy', 'immigration', 'inflation', 'trade_tariffs', 'foreign_policy', 'healthcare', 'crime']
issue_suffixes = ['econ', 'imm', 'infl', 'trade', 'fp', 'healthcare', 'crime']

for iss, suf in tqdm(zip(issues, issue_suffixes)):
    worksheet = spreadsheet.worksheet(f"approval_stats_{suf}")
    worksheet.clear()

    issue_df = pres_issue[pres_issue['issue'] == iss]
    set_with_dataframe(worksheet, issue_df.reset_index())

7it [00:06,  1.15it/s]


In [19]:
## Net issue approval
net_issues = pd.pivot_table(pres_issue[pres_issue['issue'].isin(issues)], 
                            values=['net', 'net_lower_ci', 'net_upper_ci'], index=['end_date'], columns=['issue'],
                           aggfunc="first").reset_index()
net_issues.columns = ['_'.join(col).strip() for col in net_issues.columns.values]
net_issues.head()

,end_date_,net_crime,net_economy,net_foreign_policy,net_healthcare,net_immigration,net_inflation,net_trade_tariffs,net_lower_ci_crime,net_lower_ci_economy,...,net_lower_ci_immigration,net_lower_ci_inflation,net_lower_ci_trade_tariffs,net_upper_ci_crime,net_upper_ci_economy,net_upper_ci_foreign_policy,net_upper_ci_healthcare,net_upper_ci_immigration,net_upper_ci_inflation,net_upper_ci_trade_tariffs
0,2025-01-21,0.0,7.639412,3.639412,0.000000,9.639412,-4.360588,-0.360588,0.0,7.639412,...,9.639412,-4.360588,-0.360588,0.0,7.639412,3.639412,0.000000,9.639412,-4.360588,-0.360588
1,2025-01-22,0.0,7.639412,3.639412,0.000000,9.639412,-4.360588,-0.360588,0.0,7.639412,...,9.639412,-4.360588,-0.360588,0.0,7.639412,3.639412,0.000000,9.639412,-4.360588,-0.360588
2,2025-01-23,0.0,3.026276,3.639412,-13.247049,3.686507,-4.360588,-0.360588,0.0,-3.320965,...,-4.504132,-4.360588,-0.360588,0.0,9.373518,3.639412,-13.247049,11.877145,-4.360588,-0.360588
3,2025-01-24,0.0,3.026276,3.639412,-13.247049,3.686507,-4.360588,-0.360588,0.0,-3.320965,...,-4.504132,-4.360588,-0.360588,0.0,9.373518,3.639412,-13.247049,11.877145,-4.360588,-0.360588
4,2025-01-25,0.0,3.026276,3.639412,-13.247049,3.686507,-4.360588,-0.360588,0.0,-3.320965,...,-4.504132,-4.360588,-0.360588,0.0,9.373518,3.639412,-13.247049,11.877145,-4.360588,-0.360588


In [20]:
worksheet = spreadsheet.worksheet("net_approval_issues")
worksheet.clear()

set_with_dataframe(worksheet, net_issues.reset_index())

## Polls Display Tables

In [21]:
pres_polls['sponsors'] = '^Sponsor: ' + pres_polls['sponsors'] + '^'
pres_polls['sponsors'] = pres_polls['sponsors'].fillna('')
pres_polls = pres_polls.sort_values(['total_weight'], ascending=False)
pres_polls = pres_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
pres_polls['Sample'] = pres_polls['sample_size'].astype(int).astype(str) + ' ' + pres_polls['population']
pres_polls['Start Date'] = pd.to_datetime(pres_polls['Start Date']).astype(str)
pres_polls['End Date'] = pres_polls['End Date'].astype(str)
pres_polls = pres_polls.reset_index()
pres_polls['Pollster'] = ('<a href="' + pres_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          pres_polls['Pollster'] + '</u></a>' + pres_polls['sponsors'])
pres_polls = pres_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
pres_polls

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://www.yahoo.com/news/us/article...",2026-01-08,2026-01-12,1709 A,1.348523e-01,40.0,56.0,-16.0,-14.515435
1,"<a href=""https://poll.qu.edu/poll-release?rele...",2026-01-08,2026-01-12,1133 RV,1.235179e-01,40.0,54.0,-14.0,-10.049062
2,"<a href=""https://www.cbsnews.com/news/u-s-acti...",2026-01-05,2026-01-07,2325 A,9.090832e-02,41.0,59.0,-18.0,-16.515435
3,"<a href=""https://napolitannews.org/jobapproval...",2026-01-05,2026-01-08,2000 RV,9.062170e-02,47.0,51.0,-4.0,-14.079957
4,"<a href=""https://www.ipsos.com/en-us/americans...",2026-01-12,2026-01-13,1217 A,8.658345e-02,41.0,58.0,-17.0,-13.136713
...,...,...,...,...,...,...,...,...,...
521,"<a href=""https://pro.morningconsult.com/tracke...",2025-01-24,2025-01-26,2302 RV,2.599042e-19,52.0,44.0,8.0,3.628812
522,"<a href=""https://substack.com/inbox/post/15539...",2025-01-21,2025-01-21,742 A,2.449876e-19,49.0,36.0,13.0,8.112906
523,"<a href=""https://quantusinsights.org/f/februar...",2025-02-01,2025-02-03,1000 RV,2.296763e-19,51.9,45.3,6.6,-0.188721
524,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,2.103061e-19,47.0,41.0,6.0,9.863287


In [22]:
worksheet = spreadsheet.worksheet("polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, pres_polls)

In [23]:
issue_polls['sponsor'] = '^Sponsor: ' + issue_polls['sponsor'] + '^'
issue_polls['sponsor'] = issue_polls['sponsor'].fillna('')
issue_polls = issue_polls.sort_values(['issue', 'total_weight'], ascending=False)
issue_polls = issue_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net',
    'issue':'Issue'}, axis='columns')
issue_polls['Sample'] = issue_polls['sample_size'].astype(int).astype(str) + ' ' + issue_polls['population']
issue_polls['Start Date'] = pd.to_datetime(issue_polls['Start Date']).astype(str)
issue_polls['End Date'] = issue_polls['End Date'].astype(str)
issue_polls = issue_polls.reset_index()
issue_polls['Pollster'] = ('<a href="' + issue_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          issue_polls['Pollster'] + '</u></a>' + issue_polls['sponsor'])
issue_polls = issue_polls[issue_polls['Issue'].isin(issues)]
issue_polls['Issue'] = issue_polls['Issue'].replace({
    'economy': 'Economy',
    'immigration': 'Immigration',
    'foreign_policy': 'Foreign policy',
    'inflation': 'Inflation/prices',
    'trade_tariffs': 'Trade/tariffs',
    'healthcare': 'Healthcare',
    'crime': 'Crime'
})
issue_polls = issue_polls[['Pollster', 'Start Date', 'End Date', 'Issue', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
issue_polls

,Pollster,Start Date,End Date,Issue,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
49,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-09,2026-01-12,Trade/tariffs,2201 RV,2.390138e-02,43.0,48.0,-5.0,-17.473916
50,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-09,2026-01-12,Trade/tariffs,1602 A,2.260419e-02,37.0,56.0,-19.0,-20.518681
51,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-02,2026-01-04,Trade/tariffs,2201 RV,1.056114e-02,45.0,44.0,1.0,-11.473916
52,"<a href=""https://www.atlasintel.org/poll/usa-n...",2025-12-15,2025-12-19,Trade/tariffs,2315 A,6.411338e-03,35.0,61.0,-26.0,-27.247049
53,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-12-20,2025-12-22,Trade/tariffs,1592 A,3.179174e-03,36.0,56.0,-20.0,-21.518681
...,...,...,...,...,...,...,...,...,...,...
1206,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-23,2025-02-25,Crime,1604 A,9.763877e-14,47.0,37.0,10.0,8.481319
1207,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-16,2025-02-18,Crime,1603 A,5.632211e-14,48.0,37.0,11.0,9.481319
1208,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-09,2025-02-11,Crime,1595 A,3.256002e-14,45.0,41.0,4.0,2.481319
1209,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-02,2025-02-04,Crime,1604 A,1.907977e-14,48.0,35.0,13.0,11.481319


In [24]:
worksheet = spreadsheet.worksheet("polls_display_issues")
worksheet.clear()

set_with_dataframe(worksheet, issue_polls)